In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image
from PIL import Image

In [41]:
class VGG19(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = models.vgg19(weights='IMAGENET1K_V1')
    def forward(self, x):
        features = []
        layers = [0, 5, 10, 19, 28]
        for i,layer in enumerate(self.model.features[:30]):
            x = layer(x)
            if i in layers:
                features.append(x)
        return features

In [42]:
def Gram(A):
    B = A.view(A.shape[1],-1)
    return torch.mm(B, B.t())

In [43]:
def img_to_tensor(url):
    transform = transforms.Compose([
        transforms.Resize(size=(size,size)),
        transforms.ToTensor()
    ])
    return transform(Image.open(url)).unsqueeze(0)

In [44]:
def tensor_to_img(generated):
    tensor = generated.clone()
    tensor = tensor.squeeze()
    save_image(tensor, 'generated.png')

In [45]:
def rgba_to_rgb(url):
    img = Image.open(url)
    img2 = img.convert('RGB')
    img2.save(url)

In [46]:
class NSTLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, C, S, G):
        content_loss = torch.mean((C[3]-G[3]) ** 2)
        style_loss = 0
        for i in range(5):
            G_gram = Gram(G[i])
            S_gram = Gram(S[i])
            style_loss += lambd[i] * (torch.norm(S_gram-G_gram, p = 'fro'))
        loss = (alpha * content_loss) + (beta * style_loss)
        return loss

In [47]:
url1 = './photos/tcnj.png'
url2 = './photos/snight.png'

In [48]:
model = VGG19().eval()
size = 255
content = img_to_tensor(url1)
style = img_to_tensor(url2)
generated = content.clone().requires_grad_(True)

content_feat = model(content)
content_feat = [feat.detach() for feat in content_feat]
style_feat = model(style)
style_feat = [feat.detach() for feat in style_feat]

lr = 0.003
alpha = 100
beta = 10
lambd = [0 for i in range(5)]
lambd[0] = 1
lambd[1] = .8
lambd[2] = .7
lambd[3] = .2
lambd[4] = .1
criterion = NSTLoss()
optimizer = optim.Adam([generated], lr=lr)

epochs = 501

for epoch in range(epochs):
    generated_feat = model(generated)
    optimizer.zero_grad()
    loss = criterion(content_feat, style_feat, generated_feat)
    loss.backward()
    optimizer.step()

    if epoch % 250 == 0:
        print(f'Epoch: {epoch} | Loss: {loss}')
        tensor_to_img(generated)

Epoch: 0 | Loss: 702503.0
Epoch: 250 | Loss: 43222.3359375
Epoch: 500 | Loss: 31008.25
